In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append("../src")

from config import url, file_name
from data.load_data import get_data


df = get_data(url, file_name)

In [2]:
def mainbranch_cleaned(df):
    
    map_dict = { 
        'I am a developer by profession':'Developer',
        'I am not primarily a developer, but I write code sometimes as part of my work/studies':'Coder',
        'I code primarily as a hobby':'Amateur',
        'I am learning to code':'Learner',
        'I used to be a developer by profession, but no longer am':'Ex-Developer',
        'None of these':'Other'
    }
    
    if 'MainBranch' in df.columns:
        MainBranch = df.MainBranch.map(map_dict, na_action='ignore')
        return MainBranch
    else:
        return pd.Series(index=df.index, name="Mainbranch")

In [3]:
def age_cleaned(df):
      
    if 'Age' in df.columns:
        Age = df.Age.apply(lambda x: np.nan if x=='Prefer not to say' else x)
        return Age
    else:
        return pd.Series(index=df.index, name="Age")

In [4]:
def employment_cleaned(df):
    
    if 'Employment' in df.columns:
        fltr = df.Employment.str.split(';', expand=False).apply(lambda x: 0 if x!=x  else len(x))==1
        df.loc[~fltr, 'Employment'] = np.nan
        Employment = df.Employment.apply(lambda x: np.nan if x=='I prefer not to say' else x)
        return Employment        
    else:
        return pd.Series(index=df.index, name="Employment")

In [5]:
def country_cleaned(df):
    
    if 'Country' in df.columns:
        Country = df.Country.apply(lambda x: 'Vietnam' if x=='Viet Nam' else x)
        return Country        
    else:
        return pd.Series(index=df.index, name="Country")
    

In [6]:
def edlevel_cleaned(df):
    
    if 'EdLevel' in df.columns:
        EdLevel = df.EdLevel
        return EdLevel
    else:
        return pd.Series(index=df.index)

In [7]:
def age_yearscode_cleaned(df):
    
    if 'Age' in df.columns and 'YearsCodePro' in df.columns and 'YearsCode' in df.columns:
        
        Age = df["Age"].apply(lambda x: np.nan if x=='Prefer not to say' else x)

        experience_pro = df.YearsCodePro.apply(lambda x: '0' if x=='Less than 1 year' else x) \
                        .apply(lambda x: '51' if x=='More than 50 years' else x) \
                        .apply(lambda x: int(x) if x==x else x)

        experience_tot = df.YearsCode.apply(lambda x: '0' if x=='Less than 1 year' else x) \
                        .apply(lambda x: '51' if x=='More than 50 years' else x) \
                        .apply(lambda x: int(x) if x==x else x)

        YearsCodePro = experience_pro
        YearsCodePro.loc[experience_tot-experience_pro < 0] = np.nan

        YearsCode = experience_tot
        YearsCode.loc[experience_tot-experience_pro < 0] = np.nan
        
        Age_boundaries = {
            '18-24 years old':(18,24), 
            '25-34 years old':(25,34), 
            '45-54 years old':(45,54),
            '35-44 years old':(35,44), 
            'Under 18 years old':(13, 17), 
            '55-64 years old': (55, 64),
            '65 years or older': (65, 90)
        }

        CodeInterval = df["Age"].map(Age_boundaries).apply(lambda x: (x[0]-6, x[1]-6) if x==x else x)

        tmp = pd.concat([Age, YearsCode, YearsCodePro, CodeInterval], 
                        keys=["Age", "YearsCode", "YearsCodePro", "CodeInterval"],  axis=1)

        fltr = tmp.apply(lambda x: x[1]<x[3][1] if x[3]==x[3] and x[1]==x[1] else False,  axis=1)

        tmp.loc[~fltr, :] = np.nan
        
        return tmp.loc[:, ["Age", "YearsCode", "YearsCodePro"]]
        
    else:
        return pd.DataFrame(index=df.index, columns = ["Age", "YearsCode", "YearsCodePro"])
    

In [8]:
def workexp_cleaned(df):
    
    Age = age_yearscode_cleaned(df)["Age"]
    
    if 'WorkExp' in df.columns:
                
        Age_boundaries = {
            '18-24 years old':(18,24), 
            '25-34 years old':(25,34), 
            '45-54 years old':(45,54),
            '35-44 years old':(35,44), 
            'Under 18 years old':(13, 17), 
            '55-64 years old': (55, 64),
            '65 years or older': (65, 90)
        }

        AgeInterval = Age.map(Age_boundaries).apply(lambda x: (x[0], x[1]) if x==x else x)

        tmp = pd.concat([Age, df.WorkExp, AgeInterval ], 
                        keys=["Age", "WorkExp", "AgeInterval"],  axis=1)

        fltr = tmp.apply(lambda x: x[1]<x[2][0] if x[1]==x[1] and x[2]==x[2] else False,  axis=1)

        tmp.loc[~fltr, "WorkExp"] = np.nan
        
        return tmp.loc[:, ["WorkExp"]]
        
    else:
        return pd.DataFrame(index=df.index, columns = ["WorkExp"])

In [9]:
def dataset_cleaned(df):
    
    MainBranch = mainbranch_cleaned(df)
    AgeYearsCode = age_yearscode_cleaned(df)
    Age = AgeYearsCode["Age"]
    Employment = employment_cleaned(df)
    RemoteWork = df["RemoteWork"]
    EdLevel = edlevel_cleaned(df)
    YearsCode = AgeYearsCode["YearsCode"]
    YearsCodePro = AgeYearsCode["YearsCodePro"]
    LanguageHaveWorkedWith = df["LanguageHaveWorkedWith"]
    DevType = df["DevType"]
    Country = country_cleaned(df)
    ConvertedCompYearly = df["ConvertedCompYearly"]
    ICorPM = df["ICorPM"]
    WorkExp = workexp_cleaned(df)
    Industry = df["Industry"]
    
    df_cleaned = pd.concat([MainBranch, Age, Employment, RemoteWork, EdLevel, YearsCode, YearsCodePro, 
                      LanguageHaveWorkedWith, DevType, Country, ConvertedCompYearly, ICorPM, WorkExp, Industry],
                    axis=1)
    df_cleaned = df_cleaned.drop_duplicates().dropna(subset=["ConvertedCompYearly"])
    
    return df_cleaned
                    

In [10]:
cdf = dataset_cleaned(df)
cdf

,MainBranch,Age,Employment,RemoteWork,EdLevel,YearsCode,YearsCodePro,LanguageHaveWorkedWith,DevType,Country,ConvertedCompYearly,ICorPM,WorkExp,Industry
1,Developer,25-34 years old,"Employed, full-time",Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",18.0,9.0,HTML/CSS;JavaScript;Python,"Senior Executive (C-Suite, VP, etc.)",United States of America,285000.0,People manager,10.0,"Information Services, IT, Software Development..."
2,Developer,45-54 years old,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",27.0,23.0,Bash/Shell (all shells);Go,"Developer, back-end",United States of America,250000.0,Individual contributor,23.0,"Information Services, IT, Software Development..."
3,Developer,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)","Bachelor’s degree (B.A., B.S., B.Eng., etc.)",12.0,7.0,Bash/Shell (all shells);HTML/CSS;JavaScript;PH...,"Developer, front-end",United States of America,156000.0,Individual contributor,7.0,NaN
4,Developer,25-34 years old,NaN,Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",6.0,4.0,HTML/CSS;JavaScript;TypeScript,"Developer, full-stack",Philippines,23456.0,Individual contributor,6.0,Other
5,Developer,35-44 years old,"Employed, full-time",Remote,Some college/university study without earning ...,21.0,21.0,Bash/Shell (all shells);HTML/CSS;JavaScript;Ru...,"Developer, back-end",United Kingdom of Great Britain and Northern I...,96828.0,Individual contributor,22.0,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89162,Developer,35-44 years old,"Employed, full-time",Remote,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",30.0,24.0,C;C++,"Developer, back-end",Brazil,50719.0,People manager,25.0,"Information Services, IT, Software Development..."
89166,Developer,18-24 years old,NaN,In-person,Some college/university study without earning ...,7.0,2.0,Assembly;Bash/Shell (all shells);C;C#;C++;Java...,"Developer, full-stack",Armenia,16917.0,Individual contributor,3.0,Financial Services
89167,Developer,18-24 years old,"Employed, full-time",Remote,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",6.0,2.0,Dart;HTML/CSS;JavaScript;TypeScript,"Developer, mobile",India,15752.0,Individual contributor,2.0,Retail and Consumer Services
89170,Developer,25-34 years old,"Employed, full-time","Hybrid (some remote, some in-person)","Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",18.0,9.0,C#;F#;HTML/CSS;JavaScript;SQL;TypeScript,Other (please specify):,France,64254.0,Individual contributor,9.0,Other
